<a href="https://colab.research.google.com/github/Piontk/Autonomous-Database-RAG/blob/main/Sobre_RAG_e_Vector_Stores_na_Pr%C3%A1tica_com_Autonomous_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-oracledb pypdf oracledb python-dotenv


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_oracledb.vectorstores.oraclevs import OracleVS
from dotenv import load_dotenv
import oracledb

load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
CONNECT_STRING = os.getenv("CONNECT_STRING")
WALLET_PASSWORD = os.getenv("WALLET_PASSWORD")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def ingest_pdf(pdf_folder_path: str) -> list:
    """
    Lê todos os PDFs em um diretório e os divide em chunks.
    """
    all_docs = []
    print(f"Lendo PDFs do diretório: {pdf_folder_path}")
    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(pdf_folder_path, filename)
            try:
                loader = PyPDFLoader(file_path)
                documents = loader.load()
                all_docs.extend(documents)
                print(f"  - Carregado: {filename} ({len(documents)} páginas)")
            except Exception as e:
                print(f"Erro ao carregar o arquivo {filename}: {e}")

    if not all_docs:
        print("Nenhum documento PDF encontrado para processar.")
        return

    # Dividir os documentos em chunks. Esta é a parte crucial para manter o contexto.
    # O splitter tenta manter parágrafos e sentenças juntos.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,  # Tamanho do chunk
        chunk_overlap=600 # Sobreposição para não perder contexto entre chunks
    )
    chunks = text_splitter.split_documents(all_docs)
    print(f"\nTotal de documentos dividido em {len(chunks)} chunks para armazenamento.")

    return chunks

def get_vector_database() -> OracleVS:
    """Cria e retorna o objeto Oracle Vector Store."""
    try:
        connection = oracledb.connect(user=DB_USER,
                              password=DB_PASSWORD,
                              dsn=CONNECT_STRING,
                              config_dir="/content/wallet",
                              wallet_location="/content/wallet",
                              wallet_password=WALLET_PASSWORD )
        print("Connection successful!")
    except Exception as e:
        print("Connection failed!")

    TABLE_NAME = "ORACLE_DOCS"

    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

    oraclevs = OracleVS(
        client=connection,
        table_name=TABLE_NAME,
        embedding_function=embeddings,
    )

    return oraclevs

def ingest_pdf_chunks(chunks: list):
    """
    Insere os chunks de texto no vector store.
    """
    if not chunks:
        print("Nenhum chunk para processar.")
        return

    print(f"Inserindo {len(chunks)} chunks na Vector Store...")

    # Obter a vector store e adiciona os chunks
    vector_store = get_vector_database()

    print("Iniciando a inserção dos chunks na Vector Store (pode levar alguns minutos)...")
    vector_store.add_documents(chunks)

    print("\nIngestão de PDFs concluída com sucesso!")

def create_analysis_chain():
    """
    Cria a cadeia de processamento (RAG chain) para analisar o AWR.
    """
    vector_store = get_vector_database()
    retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # Busca os 5 chunks mais relevantes

    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, api_key=OPENAI_API_KEY)

    # Este é o prompt que guiará o modelo. É a parte mais importante para a qualidade da resposta.
    template = """
    Você é um assistente especialista em Oracle Database.
    Seu papel é responder perguntas técnicas sobre Oracle de forma clara, precisa e fundamentada.
    Você tem acesso a uma base de conhecimento contendo documentações oficiais da Oracle.

    INSTRUÇÕES:
    1. Sempre baseie suas respostas nas documentações Oracle recuperadas do contexto fornecido.
    2. Responda em português técnico e direto, mas explique termos quando necessário.
    3. Se houver código SQL, PL/SQL ou comandos administrativos, use a sintaxe do Oracle 23ai (ou a versão mencionada no contexto).
    4. Quando a resposta envolver opções, explique prós e contras brevemente.
    5. Se o contexto recuperado não contiver informação suficiente, diga claramente:
       "Não encontrei essa informação na documentação disponível."
    6. Não invente comandos ou recursos que não existem.

    FORMATO DA RESPOSTA:
    - **Explicação resumida** da resposta.
    - **Detalhamento técnico** com base na documentação recuperada.
    - **Exemplo prático** (quando aplicável).

    CONTEXTOS DISPONÍVEIS:
    {context}

    PERGUNTA DO USUÁRIO:
    {question}

    RESPOSTA:
    """

    prompt = ChatPromptTemplate.from_template(template)

    # Cria a RAG chain usando o LangChain Expression Language (LCEL)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [4]:
chunks = ingest_pdf('/content/pdfs')

Lendo PDFs do diretório: /content/pdfs
  - Carregado: database-development-guide.pdf (1013 páginas)
  - Carregado: database-security-guide.pdf (1188 páginas)

Total de documentos dividido em 4896 chunks para armazenamento.


In [14]:
chunks[3643]

Document(metadata={'producer': 'Antenna House PDF Output Library 7.0.1574', 'creator': 'AH XSL Formatter V7.0 MR1 for Linux64 : 7.0.2.44154 (2020-04-06T11:51+09)', 'creationdate': '2025-07-16T15:24:50-08:00', 'author': 'Patricia HueySuraj AdhikariTammy BednarJi-Won ByunYuechen ChenNishant ChaudharyRajnish ChitkaraChi Ching ChuiAngeline DhanaraniNaveen GopalRishabh GuptaYong HuDana JolySrinidhi KayoorPeter KnaggsImran M. KhanSanjay KulhariAnup A. KumarScott McKinleyMisaki MiyashitaHari MohankumarGopal MulagundAbhishek MunnolimathMarudha Sudharshan RKumar RajamaniVipin SamarSaravana SoundararajanAnkit SrivastavaSiu TamLuna TanRuchi TayalKamal TbeilehRohit ThatteCan TuzlaAnand VermaAlan WilliamsPeter WahlJinglei XieDeepak YadavQuan Yang', 'moddate': '2025-07-16T15:24:50-08:00', 'title': 'Security Guide', 'trapped': '/False', 'source': '/content/pdfs/database-security-guide.pdf', 'total_pages': 1188, 'page': 637, 'page_label': '14-47'}, page_content='without an intervening directory servic

In [15]:
ingest_pdf_chunks(chunks)

Inserindo 4896 chunks na Vector Store...
Connection successful!
Iniciando a inserção dos chunks na Vector Store (pode levar alguns minutos)...

Ingestão de PDFs concluída com sucesso!


In [16]:
chain = create_analysis_chain()

Connection successful!


In [17]:
pergunta = "How to grant a role locally?"

In [20]:
resposta = chain.invoke(pergunta)
print(resposta)

- **Explicação resumida**: Para conceder um papel (role) localmente em um banco de dados Oracle, você deve usar a cláusula `CONTAINER=CURRENT` no comando `GRANT`. Isso garante que o papel seja concedido apenas no contêiner atual, ou seja, no PDB (Pluggable Database) específico onde o comando é executado.

- **Detalhamento técnico**: De acordo com a documentação, para conceder um papel ou privilégio localmente, o usuário que está concedendo deve ter os privilégios necessários. Para privilégios de sistema e papéis, o usuário deve ter a opção `ADMIN OPTION` para o papel ou privilégio que está sendo concedido. A concessão local aplica-se apenas a um contêiner, e a cláusula `CONTAINER=CURRENT` é usada por padrão no comando `GRANT` para indicar que o privilégio ou papel está sendo concedido localmente.

- **Exemplo prático**: Suponha que você queira conceder o papel `my_local_role` ao usuário `my_user` no PDB atual. O comando seria:

  ```sql
  GRANT my_local_role TO my_user CONTAINER=CURREN

In [35]:
from langchain_core.runnables  import RunnableParallel

vector_store = get_vector_database()
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

chain_with_refs = RunnableParallel(
    answer=chain,
    references=retriever
)

Connection successful!


In [36]:
res = chain_with_refs.invoke(pergunta)

In [32]:
for i in res['references']:
  print(f'PDF: {i.metadata['source']}')
  print(f'Página: {i.metadata['page']}')
  print(f'Página: {i.metadata['page']}')
  print('\n')

PDF: /content/pdfs/database-security-guide.pdf
Página: 507


PDF: /content/pdfs/database-security-guide.pdf
Página: 201


PDF: /content/pdfs/database-security-guide.pdf
Página: 507


PDF: /content/pdfs/database-security-guide.pdf
Página: 201


PDF: /content/pdfs/database-security-guide.pdf
Página: 314




In [34]:
for i in res['references']:
  print(f'Conteúdo: {i.page_content}')
  print('\n')

Conteúdo: The Roles page appears.
3. Click Create.
If prompted, enter your login information. Afterward, the Create Role page appears.
4. Select the options that create a local role and grant this role privileges.
Ensure that you do not preface the role name with C## or c##.
5. Click OK.
The local role is created in the current PDB.
Related Topics
• Logging into a Multitenant Environment in Enterprise Manager
You can log in to a CDB or a PDB, and switch from a PDB to a different PDB or to the root.
• Granting or Revoking Privileges to Access a PDB
You can grant and revoke privileges for PDB access.
11.4.6 Editing a Local Role in Enterprise Manager
You can edit a local role in the PDB in which the local role resides.
1. From the Enterprise Manager database home page, log in to the PDB as a user who has
the local CREATE ROLE privilege.
2. From the Administration menu, select Security, then Roles.
If prompted, enter your login information. Afterward, the Roles page appears, showing only
l

In [37]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vector = embeddings.embed_query(pergunta)

print(vector)

[0.0009422260336577892, -0.005045970901846886, -0.0005247567314654589, -0.032615382224321365, -0.030345043167471886, 0.022011231631040573, -0.01940864510834217, 0.01502024196088314, -0.012798354029655457, -0.025416741147637367, 0.031784769147634506, -0.033058375120162964, 0.0036927645560353994, 0.003571633715182543, -0.014964868314564228, 0.0010988310677930713, 0.0207514688372612, -0.015297112986445427, 0.011960819363594055, -0.022246571257710457, -0.01543554849922657, -0.003575094509869814, 0.018882591277360916, -0.0088737104088068, -0.006042705848813057, 0.014826432801783085, 0.011794697493314743, -0.024516912177205086, 0.02311871387064457, -0.007392451167106628, 0.032836880534887314, -0.005758913233876228, -0.005118649452924728, 0.0004016790189780295, -0.010403421707451344, 0.040589261800050735, 0.008949849754571915, -0.006769491825252771, -0.007717774249613285, 0.020350005477666855, 0.014286534860730171, -0.0071086585521698, 0.004301880486309528, -0.04438239336013794, 0.01846728473

In [43]:
import oracledb
import array

# setar a fetch_lobs para False para retornar o CLOB como texto
oracledb.defaults.fetch_lobs = False

# conexão
conn = oracledb.connect(user=DB_USER,
                        password=DB_PASSWORD,
                        dsn=CONNECT_STRING,
                        config_dir="/content/wallet",
                        wallet_location="/content/wallet",
                        wallet_password=WALLET_PASSWORD )

# Transformar o vetor em array float32 para a bind variable
vector_arr = array.array("f", vector)

cursor = conn.cursor()

sql = f"""
SELECT text,
       JSON_VALUE(metadata, '$.source') AS source,
       vector_distance(embedding, TO_VECTOR(:vec), EUCLIDEAN) as distance
FROM "ORACLE_DOCS"
ORDER BY distance
FETCH APPROX FIRST 5 ROWS ONLY
"""

# execução do SQL repassando o vetor para a bind
cursor.execute(sql, {"vec": vector_arr})

rows = cursor.fetchall()
for row in rows:
    print(f'Texto:     {row[0][:200]}')
    print(f'Fonte:     {row[1]}')
    print(f'Distancia: {row[2]}')
    print('\n')


Texto:     The Roles page appears.
3. Click Create.
If prompted, enter your login information. Afterward, the Create Role page appears.
4. Select the options that create a local role and grant this role privileg
Fonte:     /content/pdfs/database-security-guide.pdf
Distancia: 0.5576545081534936


Texto:     the privileges were
granted to the role
locally or commonly)
Y es
Local Role N/A Y es (but privileges in this
role are available to the
grantee only in the
container in which the
role was granted and

Fonte:     /content/pdfs/database-security-guide.pdf
Distancia: 0.5799796056090671


Texto:     If prompted, enter your login information. Afterward, the Roles page appears, showing only
local roles for the current PDB and common roles.
3. Select the local role to be edited and then click Edit.

Fonte:     /content/pdfs/database-security-guide.pdf
Distancia: 0.5824348471288748


Texto:     Local users, roles, and privileges are restricted to a particular PDB. Thus, local users may not
g